In [ ]:
import pandas as pd
import numpy as np

# Memuat dataset
data = pd.read_csv("/content/drive/MyDrive/STKI/data.csv")
id_wilayah = pd.read_csv("/content/drive/MyDrive/STKI/Country Codes Alpha-2 Alpha-3.csv")


In [ ]:
#cleaning data untuk menghilangkan missing value
data = data[data['title'].isna()==False]
data['url'] = data['url'].fillna('https://www.netflix.com/')
data['genres'] = data['genres'].fillna('Other')
data['imdbId'] = data['imdbId'].fillna('Unknown')
data['imdbAverageRating'] = data['imdbAverageRating'].fillna(data['imdbAverageRating'].mean())
data['imdbNumVotes'] = data['imdbNumVotes'].fillna(data['imdbNumVotes'].mean())

<ipython-input-5-463a84c53928>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['url'] = data['url'].fillna('https://www.netflix.com/')
<ipython-input-5-463a84c53928>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres'] = data['genres'].fillna('Other')
<ipython-input-5-463a84c53928>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [ ]:
# teknik untuk mencari film berdasarkan parameter dan mengurutkannya
def search_movies(type=None, genre=None, year=None, country=None):
    alpha = id_wilayah[id_wilayah['country'] == country.capitalize()]
    input_wilayah = alpha['alpha2'].tolist()[0]
    print(input_wilayah)
    filtered_data = data
    if type:
        filtered_data = filtered_data[filtered_data['type'] == type]
    if genre:
        genres_list = [genree.strip().lower() for genree in genre.split(',')] # Filter baris data di mana genre sesuai
        filtered_data = filtered_data[filtered_data['genres'].apply(lambda x: isinstance(x, str) and any(g in [gen.strip().lower() for gen in x.split(',')] for g in genres_list))]
    if year:
        years_list = [float(year.strip()) for year in year.split(',')] # Filter baris data di mana tahun sesuai
        filtered_data = filtered_data[filtered_data['releaseYear'].isin(years_list)]
    if country:
        filtered_data = filtered_data[filtered_data['availableCountries'].str.contains(input_wilayah, case=False, na=False)]

    sorted_data = filtered_data.sort_values(by='imdbAverageRating', ascending=False)

    return sorted_data

# type = 'movie'
# genre = 'action'
# year = '2018,2022'
# country = 'indonesia'
type = input('type movie dan tv : ')
genre = input('genrenya apa : ')
year = input('tahun rilis : ')
country = input('id wilayah : ')
jumlah_rekomendasi = int(input('masukan jumlah film yang akan direkomendasi'))

sorted_movies = search_movies(type=type,year=year,country=country,genre=genre).head(jumlah_rekomendasi)
print(sorted_movies[['title','type', 'genres', 'releaseYear', 'imdbAverageRating']].to_string(index=False))


type movie dan tv : movie
genrenya apa : horror
tahun rilis : 2005
id wilayah : Indonesia
masukan jumlah film yang akan direkomendasi20
ID
                            title  type                     genres  releaseYear  imdbAverageRating
                      Constantine movie    Action, Fantasy, Horror       2005.0                7.0
                           Hostel movie                     Horror       2005.0                5.9
                         The Maid movie     Drama, Horror, Mystery       2005.0                5.7
Dominion: Prequel to the Exorcist movie    Drama, Horror, Thriller       2005.0                5.1
                             Kaal movie Adventure, Horror, Mystery       2005.0                4.7


In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


data['genres'] = data['genres'].str.lower()
# Menggunakan TF-Iata Vectorizer untuk merepresentasikan genre sebagai vektor
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(', '))
tfidf_matrix = vectorizer.fit_transform(data['genres'])

# Menghitung frekuensi genre menggunakan vektor TF-IDF
genre_frequencies = tfidf_matrix.sum(axis=0).A1
genre_terms = vectorizer.get_feature_names_out()
genre_counts = dict(zip(genre_terms, genre_frequencies))

# Menampilkan frekuensi genre

# print("Frekuensi Genre:")
# for genre, count in genre_counts.items():
#     print(f"{genre}: {count:.2f}")


#permintaan pencarian

print("cari film berdasarkan kombinasi genre menggunakan tfidf atau vector space model")
print("-------------")
print("\n")
query = input('masukan genre apa saja yang ingin dicari : ').lower()
jumlah_hasil = int(input('jumlah hasil yang diinginkan : '))

# Mengonversi permintaan pencarian menjadi vektor TF-IDF
query_vector = vectorizer.transform([query])

# Menghitung cosine similarity antara permintaan dan film
cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()

# Menampilkan hasil pencarian berdasarkan kemiripan
results = cosine_sim.argsort()[::-1][:50]

print("\nHasil Pencarian:")
for idx in results:
    print(f"Title: {data.loc[idx, 'title']}, Genre: {data.loc[idx, 'genres']} (Skor: {cosine_sim[idx]:.4f})")


cari film berdasarkan kombinasi genre menggunakan tfidf atau vector space model
-------------


masukan genre apa saja yang ingin dicari : horror
jumlah hasil yang diinginkan : 10

Hasil Pencarian:
Title: The Chanting 3, Genre: horror (Skor: 1.0000)
Title: The Revenge of Robert the Doll, Genre: horror (Skor: 1.0000)
Title: Lust for a Vampire, Genre: horror (Skor: 1.0000)
Title: Bring Back the Dead, Genre: horror (Skor: 1.0000)
Title: Two Witches, Genre: horror (Skor: 1.0000)
Title: The Evil Dead, Genre: horror (Skor: 1.0000)
Title: Follow Me to Hell, Genre: horror (Skor: 1.0000)
Title: Jengka, Genre: horror (Skor: 1.0000)
Title: Pwera usog, Genre: horror (Skor: 1.0000)
Title: The Crazies, Genre: horror (Skor: 1.0000)
Title: 40 Hari Bangkitnya Pocong, Genre: horror (Skor: 1.0000)
Title: Aura, Genre: horror (Skor: 1.0000)
Title: Hostel: Part III, Genre: horror (Skor: 1.0000)
Title: Amityville Playhouse, Genre: horror (Skor: 1.0000)
Title: Nail Demon, Genre: horror (Skor: 1.0000)
Title: T